In [1]:
from safetensors.torch import load_file
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from sagemaker.pytorch import PyTorch
import numpy as np
import torch.nn as nn

2025-02-21 23:50:16.369601: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-21 23:50:16.384462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-21 23:50:16.403775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-21 23:50:16.409660: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 23:50:16.423161: I tensorflow/core/platform/cpu_feature_guar

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from safetensors.torch import load_file
import torch
from sagemaker.serve.mode.function_pointers import Mode
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.serve import InferenceSpec
import boto3
from sagemaker.serve import CustomPayloadTranslator
import torch
from torchvision.transforms import transforms
from sagemaker.serve.spec.inference_spec import InferenceSpec
from transformers import pipeline
from sagemaker.serve import ModelServer
from sagemaker.pytorch import PyTorchModel
import sagemaker
from pathlib import Path
import numpy as np
import io
import os
sagemaker_session = Session()
region = boto3.Session().region_name

# get execution role
# please use execution role if you are using notebook instance or update the role arn if you are using a different role
execution_role = get_execution_role() if get_execution_role() is not None else "your-role-arn"

In [3]:
%%capture
!pip install -U accelerate

In [4]:
import unsloth

INFO:datasets:PyTorch version 2.5.1+cu121 available.
INFO:datasets:TensorFlow version 2.17.0 available.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Loading the Finetuned Model
SAVED_MODEL is the lora huggingface repo of the model that has been fine-tuned. INFER_MODEL is the huggingface repo of the model that has the lora weights merged with the base weights for inferring.

In [5]:
SAVED_MODEL = "Alexis-Az/Story-Generation-LlaMA-3.1-8B-10k"
INFER_MODEL= "Alexis-Az/Story-Generation-Model"
max_seq_length = 1024

In [ ]:
adapter_model, tokenizer = unsloth.FastLanguageModel.from_pretrained(SAVED_MODEL, load_in_4bit=True)

In [ ]:
adapter_model.save_pretrained_merged("Story-Generation-LlaMA-3.1-8B-10k", tokenizer)

In [ ]:
adapter_model

In [ ]:
!ls

## Saving the Merged Model 
The model will be pushed to the INFER_MODEL huggingface repo, and will also be saved as a model .pth file for use in the container for inferrence.

In [ ]:
pth_model = AutoModelForCausalLM.from_pretrained("./Story-Generation-LlaMA-3.1-8B-10k")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./Story-Generation-LlaMA-3.1-8B-10k")

In [ ]:
pth_model.push_to_hub(
    INFER_MODEL,
    tokenizer=tokenizer,
    safe_serialization=True,
    create_pr=True,
    max_shard_size="3GB",
)

In [ ]:
tokenizer.push_to_hub(INFER_MODEL)

In [ ]:
from pathlib import Path
model_dir = "./serve"
!mkdir -p {model_dir}

In [ ]:
torch.save(pth_model.state_dict(), model_dir + '/model.pth')

In [ ]:
!ls

## Scripting the Model as a .pth TorchScript file

In [6]:
pth_model, tokenizer = unsloth.FastLanguageModel.from_pretrained(INFER_MODEL, load_in_4bit=True)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

In [8]:
unsloth.FastLanguageModel.for_inference(pth_model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [9]:
from transformers import TextStreamer

In [10]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

In [11]:
text_streamer = TextStreamer(tokenizer)
messages = [
    {"from": "human", "value": "tell me a story about a cute park in california"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")

In [12]:
_ = pth_model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

tell me a story about a cute park in california<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Once upon a time, in the heart of California, there was a charming park called "Sunset Oasis." It was nestled between two rolling hills, surrounded by a lush carpet of green grass and vibrant wildflowers that bloomed in every color of the rainbow.

The park was designed to resemble a tranquil oasis in the middle of the bustling city. A winding path made of smooth stone led visitors through the park, past a tranquil pond where ducks and swans glided effortlessly across the water.

As visitors strolled through the park, they were greeted by the sweet songs of birds and the gentle rustle of leaves in the breeze. The air was filled with the scent of blooming flowers and the warm, golden light of the California su

In [13]:
pth_model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [19]:
DUMMY_TEXT = 'tell me a story about a cute park in california'
tok_text = tokenizer.tokenize(DUMMY_TEXT)
tokens = tokenizer.encode(DUMMY_TEXT)
tok_arr = np.asarray(tokens).reshape(1, -1).astype(np.int32)
tok_arr = np.pad(tok_arr, [(0,0), (0, 1024-tok_arr.shape[1])])
tok_arr = np.repeat(tok_arr, 2, axis=0)

In [6]:
class TritonWrapper(nn.Module):
    
    def __init__(self, model_name: str):
        super().__init__()

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torchscript=True,
            device_map='cuda:0',
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            )
        self.model.to('cuda:0')
        self.model.output_hidden_states = False
                
    def forward(self, tokens_tensor):
        self.model.eval()
        o = self.model(tokens_tensor, output_hidden_states=False)
        return o[0]

In [8]:
model = TritonWrapper(INFER_MODEL)
model.eval()

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1008.00 MiB. GPU 0 has a total capacity of 14.57 GiB of which 492.75 MiB is free. Process 22303 has 14.08 GiB memory in use. Of the allocated memory 13.98 GiB is allocated by PyTorch, and 1.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
assert model.training == False
with torch.no_grad():
    tokens_tensor = torch.from_numpy(tok_arr).to('cuda:0')
    o = model(tokens_tensor)
    traced_model = torch.jit.trace(model, [tokens_tensor])
    print("traced_model done")
    torch.jit.save(traced_model, f"{model_name.split('/')[-1]}.pt")

In [35]:
def return_logits(model, inputs):
    """
    Extracts logits from the model for given inputs
    
    Args:
        model: PyTorch model instance
        inputs: Input tensor to process
        
    Returns:
        logits: Raw unnormalized scores from the final layer
    """
    
    # Forward pass through the model
    outputs = model.generate(inputs, do_sample=True, top_k=50, max_length=100, top_p=0.95, temperature=1.0)
    
    # Get logits directly from outputs
    # Note: This assumes your model returns a tuple where the first element contains the logits
    # Adjust this line based on your specific model's output structure
    logits = outputs.logits if hasattr(outputs, 'logits') else outputs
    
    return logits

In [36]:
inputs.requires_grad = False
pth_model.requires_grad = False

In [47]:
for j, p in enumerate(pth_model.parameters()):
        p.requires_grad_(True)

RuntimeError: only Tensors of floating point dtype can require gradients

In [ ]:
# Trace the model
with torch.no_grad():
    def return_logits(model, inputs):
        """
        Extracts logits from the model for given inputs
        
        Args:
            model: PyTorch model instance
            inputs: Input tensor to process
            
        Returns:
            logits: Raw unnormalized scores from the final layer
        """
        
        # Forward pass through the model
        outputs = model.generate(inputs, max_length=100)
        
        # Get logits directly from outputs
        # Note: This assumes your model returns a tuple where the first element contains the logits
        # Adjust this line based on your specific model's output structure
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs
        
        return logits
    
    model_trace = torch.jit.trace(
        lambda x: return_logits(pth_model, x),
        (inputs,)
    )

# Using Sagemaker's 'ModelBuilder' Class to Format the PyTorch Model Artifacts

## Inference Object for Handling Input/Output of Model

In [ ]:
class InferenceSpec(InferenceSpec):
    def invoke(self, input_object: object, model: object):       
        with torch.no_grad():
            output = model(input_object)
        return output
        
    def load(self, model_dir: str):
        model = AutoModelForCausalLM
        model.load_state_dict(torch.load(model_dir+'/model.pth'))
        model.eval()
        return model

inf_spec = InferenceSpec()

In [ ]:
instance_type = 'ml.g5.2xlarge'
image = image_uris.retrieve(region=region, framework='pytorch', image_scope='inference', version='2.0.0', base_framework_version='pytorch2.0.0', instance_type=instance_type)

In [ ]:
value: str = "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:"
schema = SchemaBuilder(value,
            {"generated_text": "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\\nDaniel: Hello, Girafatron!\\nGirafatron: Hi, Daniel. I was just thinking about how magnificent giraffes are and how they should be worshiped by all.\\nDaniel: You and I think alike, Girafatron. I think all animals should be worshipped! But I guess that could be a bit impractical...\\nGirafatron: That\'s true. But the giraffe is just such an amazing creature and should always be respected!\\nDaniel: Yes! And the way you go on about giraffes, I could tell you really love them.\\nGirafatron: I\'m obsessed with them, and I\'m glad to hear you noticed!\\nDaniel: I\'"})

In [ ]:
model_dir = "./serve"
local_model_dir = str(Path(model_dir).resolve())

In [ ]:
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    inference_spec=inf_spec,
    model_path=model_dir,
    role_arn=execution_role,
    image_uri=image,
    schema_builder=schema,
    env_vars={"HF_TASK":"text-generation"},
    model_server=ModelServer.TORCHSERVE
)

In [ ]:
model_builder

In [ ]:
aws_model = model_builder.build(role_arn=execution_role, sagemaker_session=sagemaker_session)

In [ ]:
source_directory = "./serve"  # Replace with the actual directory path
output_file = "model"

In [ ]:
!tar --use-compress-program="pigz --best --recursive" -cf 'model.tar.gz' './serve'

In [ ]:
import boto3

# 1. Create an S3 client
s3 = boto3.client('s3', region_name=region)  # Replace 'YOUR_REGION' with your bucket's region

# 2. Define file and bucket details
file_path = './model.tar.gz'   # Path to the local file you want to upload
bucket_name = 'unsloth-finetuned'      # Name of your S3 bucket
object_key = 'model.tar.gz'  # Desired key (filename) for the object in S3

try:
    # 3. Upload the file
    s3.upload_file(file_path, bucket_name, object_key)
    print(f"File '{file_path}' uploaded to S3 bucket '{bucket_name}' as '{object_key}'")
except Exception as e:
    print(f"Error uploading file: {e}")
